In [1]:
Id=[]
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/mushrooms-images-classification-215/data/data'):
    for filename in filenames:
        Id.append(os.path.join(dirname, filename))
Id[:5]

['/kaggle/input/mushrooms-images-classification-215/data/data/elfin_saddle/11.png',
 '/kaggle/input/mushrooms-images-classification-215/data/data/elfin_saddle/4.png',
 '/kaggle/input/mushrooms-images-classification-215/data/data/elfin_saddle/9.png',
 '/kaggle/input/mushrooms-images-classification-215/data/data/elfin_saddle/14.png',
 '/kaggle/input/mushrooms-images-classification-215/data/data/elfin_saddle/1.png']

In [2]:
train=pd.DataFrame()
train=train.assign(filename=Id)
train.head()

,filename
0,/kaggle/input/mushrooms-images-classification-...
1,/kaggle/input/mushrooms-images-classification-...
2,/kaggle/input/mushrooms-images-classification-...
3,/kaggle/input/mushrooms-images-classification-...
4,/kaggle/input/mushrooms-images-classification-...


In [3]:
train['label']=train['filename']
train['label']=train['label'].str.replace('/kaggle/input/mushrooms-images-classification-215/data/data/','')
train.head()

,filename,label
0,/kaggle/input/mushrooms-images-classification-...,elfin_saddle/11.png
1,/kaggle/input/mushrooms-images-classification-...,elfin_saddle/4.png
2,/kaggle/input/mushrooms-images-classification-...,elfin_saddle/9.png
3,/kaggle/input/mushrooms-images-classification-...,elfin_saddle/14.png
4,/kaggle/input/mushrooms-images-classification-...,elfin_saddle/1.png


In [4]:
train['label'] = train['label'].str.split('/').str[0]
train.head()

,filename,label
0,/kaggle/input/mushrooms-images-classification-...,elfin_saddle
1,/kaggle/input/mushrooms-images-classification-...,elfin_saddle
2,/kaggle/input/mushrooms-images-classification-...,elfin_saddle
3,/kaggle/input/mushrooms-images-classification-...,elfin_saddle
4,/kaggle/input/mushrooms-images-classification-...,elfin_saddle


In [5]:
count=(train.label.value_counts()).to_frame()
count.reset_index(inplace=True)
count.columns=['label','count']
count

,label,count
0,deadly_fibrecap,16
1,elfin_saddle,15
2,the_prince,15
3,amethyst_deceiver,15
4,red_cracking_bolete,15
...,...,...
210,cinnamon_bracket,9
211,devils_bolete,8
212,deadly_webcap,8
213,orange_birch_bolete,8


In [6]:
train=train[train.label!='truffles']
count=(train.label.value_counts()).to_frame()
count.reset_index(inplace=True)
count.columns=['label','count']
count

,label,count
0,deadly_fibrecap,16
1,elfin_saddle,15
2,the_prince,15
3,amethyst_deceiver,15
4,red_cracking_bolete,15
...,...,...
209,hoof_fungus,10
210,cinnamon_bracket,9
211,devils_bolete,8
212,deadly_webcap,8


In [7]:
from sklearn.model_selection import train_test_split
train,test= train_test_split( train, test_size=0.2, random_state=42,shuffle=True, stratify=train['label'])

In [8]:
count=(train.label.value_counts()).to_frame()
count.reset_index(inplace=True)
count.columns=['label','count']
count

,label,count
0,deadly_fibrecap,13
1,slimy_waxcap,12
2,dusky_puffball,12
3,beechwood_sickener,12
4,scarlet_caterpillarclub,12
...,...,...
209,cedarwood_waxcap,8
210,cinnamon_bracket,7
211,orange_birch_bolete,6
212,deadly_webcap,6


In [9]:
train=train.sample(n=len(train))
train.reset_index(inplace=True,drop=True)
train.head()

,filename,label
0,/kaggle/input/mushrooms-images-classification-...,snowy_waxcap
1,/kaggle/input/mushrooms-images-classification-...,wood_mushroom
2,/kaggle/input/mushrooms-images-classification-...,snakeskin_grisette
3,/kaggle/input/mushrooms-images-classification-...,ochre_brittlegill
4,/kaggle/input/mushrooms-images-classification-...,stump_puffball


In [10]:
test=test.sample(n=len(test))
test.reset_index(inplace=True,drop=True)
test.head()

,filename,label
0,/kaggle/input/mushrooms-images-classification-...,silverleaf_fungus
1,/kaggle/input/mushrooms-images-classification-...,shaggy_parasol
2,/kaggle/input/mushrooms-images-classification-...,terracotta_hedgehog
3,/kaggle/input/mushrooms-images-classification-...,thimble_morel
4,/kaggle/input/mushrooms-images-classification-...,smoky_bracket


In [11]:
train.shape,test.shape

((2494, 2), (624, 2))

In [12]:
import os
import cv2
import multiprocessing
from functools import partial
import pandas as pd
import uuid

def _process_df_chunk(chunk, file_col, label_col, dest_folder):
    def _process_image(file_path, label):
        try:
            # Load the image
            img = cv2.imread(file_path)
            # Check if the image size is empty
            if img is None or img.size == 0:
                print(f"Error: Could not read file {file_path}")
                return
            # Resize to 512x512
            img = cv2.resize(img, (512, 512))
            # Save the image to the folder with label name
            folder_path = os.path.join(dest_folder, str(label))
            if not os.path.exists(folder_path):
                os.makedirs(folder_path)
            file_name = os.path.basename(file_path)
            # Append a unique identifier to the file name to avoid overwriting
            name, extension = os.path.splitext(file_name)
            save_path = os.path.join(folder_path, f"{name}_{uuid.uuid4().hex}{extension}")
            cv2.imwrite(save_path, img)
        except Exception as e:
            print(f"Error: {e}")
            
    for _, row in chunk.iterrows():
        _process_image(row[file_col], row[label_col])

def resize_save_image(df, file_col, label_col, dest_folder):
    try:
        # Determine the number of processes to use
        num_processes = multiprocessing.cpu_count()

        # Split the DataFrame into chunks to process in parallel
        df_chunks = [df[i:i + num_processes] for i in range(0, len(df), num_processes)]

        # Create a partial function with fixed arguments
        func = partial(_process_df_chunk, file_col=file_col, label_col=label_col, dest_folder=dest_folder)

        # Run the function in parallel on the DataFrame chunks
        with multiprocessing.Pool(processes=num_processes) as pool:
            pool.map(func, df_chunks)
    except Exception as e:
        print(f"Error: {e}")

In [13]:
import os
newpath = r'./train' 
if not os.path.exists(newpath):
    os.makedirs(newpath)

In [14]:
resize_save_image(train, file_col='filename', label_col='label' ,dest_folder='./train')

In [15]:
newpath = r'./test' 
if not os.path.exists(newpath):
    os.makedirs(newpath)

In [16]:
resize_save_image(test, file_col='filename', label_col='label' ,dest_folder='./test')